In [1]:
import sys
import os
import re
import json
import gzip
from io import BytesIO
from random import sample

from pyspark.accumulators import AccumulatorParam
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import input_file_name


In [2]:
bucket_name = 's3a://big-data-sets-bjh/'
person = sqlCtx.read.parquet(bucket_name + 'dataworld/person/state=*/year=14/')
ontology = sqlCtx.read.parquet(bucket_name + 'dataworld/ontology')
ontology.createOrReplaceTempView('ontology')
person.createOrReplaceTempView('person')
person.count()
top_20 = [p.asDict() for p in person.take(20)]
person_cols = list(top_20[0].keys())
ont = [o.asDict() for o in ontology.collect()]
person_ont = [o for o in ont if o['name'] in person_cols]


3109172

In [62]:
# person_cols

In [60]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
 
# Define the UDF
def replace(data, string, replacement):
    return data.replace(string, replacement)

udf_replace = udf(replace, StringType())
sqlCtx.registerFunction('replace', replace, StringType())
q = '''
SELECT p.acs_hasPrivateHealthInsurance, o.state AS state from person p JOIN 
(SELECT data.`rdfs:label` as state, name from ontology where name like "acs_State%") o
ON p.acs_hasState = replace(o.name, ':', '_') 
group by acs_hasPrivateHealthInsurance, state
'''

# by_state = sqlCtx.sql(state_q)
res = sqlCtx.sql(q)


0